# 第6章　座標変換／空間解析／３次元データ処理：地理空間情報システム（GIS）

本Notebookは，Interface誌特集記事第6章を読んでいただく際に，読者の手元のPython環境で試して体験していただくために公開・配布しています．詳細については，記事本文を参照ください．

## 6-1 基礎知識

### 実行環境の準備

jupyterlabのインストールを行います．
```
pip install jupyterlab
```
同様に数値計算およびプロット，GIS関連の描画，測位計算に利用するパッケージをインストールします．
```
pip install numpy, matplotlib, folium, cartopy, cssrlib
```
その後，以下のようにJupyerLabを起動します．
```
jupyter lab
```
Webブラウザが自動的に起動し，Jupyter notebookがオープンされます．

## 6-2 異なる座標系間の座標値を変換するヘルマート変換

リスト1(a)

In [ ]:
import numpy as np
from cssrlib.gnss import pos2ecef, ecef2pos


def conv_cord(ph, lat, lon, alt, t=0, dph=None, tref=2015.0):
    c = np.array(ph[0:3]*1e-3)
    d = ph[3]*1e-9
    r = np.array(np.deg2rad(ph[4:7]*1e-3/3600))

    if dph is not None:
        dt = t-tref
        c += dph[0:3]*1e-3*dt
        d += dph[3]*1e-9*dt
        r += np.array(np.deg2rad(dph[4:7]*dt*1e-3/3600))

    x = pos2ecef([lat, lon, alt], isdeg=True)
    R = np.array([[0, -r[2], r[1]], [r[2], 0, -r[0]], [-r[1], r[0], 0]])
    xp = (1+d)*x + R@x + c
    print(f"x'-x = {xp-x}")
    lat, lon, alt = ecef2pos(xp)
    return np.rad2deg(lat), np.rad2deg(lon), alt

リスト1（b）

In [ ]:
# ITRF94 to ETRF2020
# http://etrs89.ensg.ign.fr/pub/EUREF-TN-1-Mar-04-2024.pdf
# T1[mm], T2[mm], T3[mm], D[ppb], R1[mas], R2[mas], R3[mas]
ph = np.array([-6.5, 3.0, 77.9, -3.98, 2.236, 13.494, -19.938])
dph = np.array([-0.1, 0.6, 3.1, -0.12, 0.086, 0.519, -0.773])  # rate [?/yr]

lat1, lon1, alt1 = 52.3759, 9.7320, 100  # ハノーファー
t1 = 2016
lat2, lon2, alt2 = conv_cord(ph, lat1, lon1, alt1, t=t1, dph=dph)
print(f"lat={lat2}, lon={lon2}, alt={alt2}")

x'-x = [ 0.385484   -0.44187066 -0.19346528]
lat=52.37589676598711, lon=9.73199264821396, alt=100.03311329521239


## 6-3 高精度測位向け…地殻変動誤差の補正

リスト2（a）

In [ ]:
import pyproj
from pyproj.transformer import TransformerGroup

pyproj.datadir.set_data_dir('D:\\tools\\OSGeo4W\\share\\proj')

リスト2（b）

In [ ]:
lat0, lon0, alt0 = 38.2682, 140.8694, 89.3  # 仙台市役所（旧日本測地系）

transformer = pyproj.Transformer.from_crs("EPSG:4301", "EPSG:6668")
lat0d, lon0d = transformer.transform(lat0, lon0)
print(f"旧日本測地系 緯度: {lat0:.6f} 経度: {lon0:.6f}")
print(f"JGD2011 緯度: {lat0d:.6f} 経度: {lon0d:.6f}")

旧日本測地系 緯度: 38.268200 経度: 140.869400
JGD2011 緯度: 38.271170 経度: 140.865969



リスト3（a）

In [ ]:
# JGD2011 (楕円体高) -> JGD2011 (標高)
trans = pyproj.Transformer.from_crs("EPSG:6667", "EPSG:6695")
_, _, alt2 = trans.transform(lat0d, lon0d, alt0)
print(f"楕円体高: {alt0:.2f}[m] 標高 {alt2:.2f}[m]")

楕円体高: 89.30[m] 標高 47.50[m]


リスト4（a）

In [ ]:
trans_group = TransformerGroup("EPSG:4301", "EPSG:6668")
trans_group.transformers

[<Transformation Transformer: pipeline>
 Description: Tokyo to JGD2011 (2)
 Area of Use:
 - name: Japan - onshore, excluding northern prefectures of 'main province' (see remarks).
 - bounds: (122.83, 20.37, 154.05, 45.54),
 <Concatenated Operation Transformer: pipeline>
 Description: Tokyo to JGD2011 (1)
 Area of Use:
 - name: Japan - northern Honshu prefectures affected by 2011 Tohoku earthquake: Aomori, Iwate, Miyagi, Akita, Yamaguta, Fukushima, Ibaraki, Tochigi, Gumma, Saitama, Chiba, Tokyo, Kanagawa, Niigata, Toyama, Ishikawa, Fukui, Yamanashi, Nagano, Gifu.
 - bounds: (135.42, 34.84, 142.14, 41.58),
 <Transformation Transformer: noop>
 Description: Ballpark geographic offset from Tokyo to JGD2011
 Area of Use:
 - name: World
 - bounds: (-180.0, -90.0, 180.0, 90.0)]

リスト5（a）

In [ ]:
lat0a, lon0a = trans_group.transformers[0].transform(lat0, lon0)
lat0b, lon0b = trans_group.transformers[1].transform(lat0, lon0)

print(f"旧日本測地系 緯度: {lat0:.6f} 経度: {lon0:.6f}")
print(f"JGD2011(地殻変動なし) 緯度: {lat0a:.6f} 経度: {lon0a:.6f}")
print(f"JGD2011(地殻変動あり) 緯度: {lat0b:.6f} 経度: {lon0b:.6f}")

旧日本測地系 緯度: 38.268200 経度: 140.869400
JGD2011(地殻変動なし) 緯度: 38.271170 経度: 140.865969
JGD2011(地殻変動あり) 緯度: 38.271162 経度: 140.866006


リスト6（a）

In [ ]:
lat1, lon1, alt1 = 38.2687, 140.8722, 89.3  # 宮城県庁

grs80 = pyproj.Geod(ellps='GRS80')  # GRS80楕円体
az, bkw_az, dist = grs80.inv(lon0, lat0, lon1, lat1)
print(f"方位 = {az:.2f} [deg] 距離 = {dist:.2f} [m]")

方位 = 77.24 [deg] 距離 = 251.24 [m]


## 6-4　最小二乗法によりヘルマート変換のパラメータ推定を行う

リスト7（a）

In [ ]:
np.random.seed(1)
pr = np.random.normal(size=7)
pr

array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
       -2.3015387 ,  1.74481176])

リスト8（a）

In [ ]:
from cssrlib.gnss import rCST

lat0, lon0, alt0 = 38.2682, 140.8694, 89.3  # 仙台市役所

dlat = np.rad2deg(300/rCST.RE_WGS84)
dlon = dlat/np.cos(np.deg2rad(lat1))
dlat_t = [dlat, 0, dlat]
dlon_t = [0, dlon, dlon]

n = 3
pos_r = np.zeros((n, 3))

for k in range(n):
    pos_r[k, :] = [lat0+dlat_t[k], lon0+dlon_t[k], alt0]

リスト9（a）

In [ ]:
import folium

m = folium.Map(location=[lat0, lon0], zoom_start=15)
folium.Marker(location=[lat0, lon0], icon=folium.Icon(color='red', icon='home')).add_to(m)
for k in range(n):
    folium.Marker(location=[pos_r[k,0], pos_r[k,1]]).add_to(m)
m

リスト10（a）

In [ ]:
pos = np.zeros((n, 3))
r = np.zeros((n, 3))
rp = np.zeros((n, 3))
for k in range(n):
    pos[k, :] = conv_cord(pr, pos_r[k, 0], pos_r[k, 1], pos_r[k, 2])
    r[k, :] = pos2ecef(pos_r[k, :], isdeg=True)
    rp[k, :] = pos2ecef(pos[k, :], isdeg=True)

x'-x = [-0.06481125 -0.05339247 -0.03486617]
x'-x = [-0.06480728 -0.05339418 -0.03487013]
x'-x = [-0.06480907 -0.05339382 -0.03486927]


リスト11（a）

In [ ]:
def skew(r):
    return np.array([[0, r[2], -r[1]], [-r[2], 0, r[0]], [r[1], -r[0], 0]])

M = np.zeros((3*n, 7))
r0 = 0
for k in range(n):
    M[0+r0:3+r0, 0:3] = np.eye(3)*1e-3
    M[0+r0:3+r0, 3] = r[k, :]*1e-9
    M[0+r0:3+r0, 4:7] = skew(np.deg2rad(r[k, :]*1e-3/3600))
    r0 += 3

b = np.reshape(rp-r, [9, 1])
p = np.linalg.inv(M.T@M)@M.T@b
p

array([[ 1.62628517],
       [-0.60860503],
       [-0.53824258],
       [-1.07205096],
       [ 0.8659201 ],
       [-2.30161331],
       [ 1.74461438]])

## 6-5　3次元位置と時刻を求める衛星測位

リスト12（a）

In [ ]:
import numpy as np
from cssrlib.gnss import pos2ecef, xyz2enu, rCST

lat0, lon0, alt0 = 38.2682, 140.8694, 89.3  # 仙台市役所（旧日本測地系）

pos0 = np.array([np.deg2rad(lat0), np.deg2rad(lon0), alt0])
r0 = pos2ecef(pos0)

nsat = 5
az_t = np.deg2rad([45, 135, 225, 315, 180])
el_t = np.deg2rad([45, 45, 45, 45, 90])
rng_t = 20000e3*np.ones(nsat)

E = xyz2enu(pos0)
xs = np.zeros((nsat, 3))
for k in range(nsat):
    s_az, c_az = np.sin(az_t[k]), np.cos(az_t[k])
    s_el, c_el = np.sin(el_t[k]), np.cos(el_t[k])
    rs = rng_t[k]*np.array([s_az*c_el, c_az*c_el, s_el])
    xs[k, :] = E.T@rs + r0

リスト12（b）

In [ ]:
np.random.seed(1)

tcb = np.random.normal(0, 0.1e-3*rCST.CLIGHT)
rm = rng_t + np.random.normal(0.0, 1.0, nsat) + tcb

リスト12（c）

In [ ]:
x = np.zeros(4)
x[0:3] = pos2ecef(np.array([90.0, 0, 0]), isdeg=True)
b = np.zeros(nsat)
H = np.zeros((nsat, 4))

for iter in range(10):
    for k in range(nsat):
        r_ = np.linalg.norm(x[0:3]-xs[k, :]) + x[3]
        H[k, 0:3] = (x[0:3]-xs[k, :])/r_
        H[k, 3] = 1
        b[k] = rm[k] - r_

    Q = np.linalg.inv(H.T@H)
    dx = Q@H.T@b
    # dx, res, _, _ = np.linalg.lstsq(H, b, None)
    x = x + dx
    m_dx = np.linalg.norm(dx)
    print(f"iter={iter+1} x={x} |dx|={m_dx}")
    if m_dx < 1e-3:
        break

p_err = E@(x[0:3]-r0)
print(f"position error (ENU) = {p_err}\n tcb[estim] = {x[3]:.3f} tcb = {tcb:.3f}")

iter=1 x=[-4180589.74731046  3401181.26747221  3410438.3804023    643416.43880721] |dx|=6175768.386201451
iter=2 x=[-3874618.25284895  3152253.75185015  3941111.31682473    50818.2113988 ] |dx|=887901.9585717699
iter=3 x=[-3889538.45846488  3164392.28299686  3928878.72264603    48705.20507356] |dx|=22892.305041050484
iter=4 x=[-3889502.30984406  3164362.87451879  3928918.84649905    48701.05519091] |dx|=61.633809526645514
iter=5 x=[-3889502.39790784  3164362.94616054  3928918.74885074    48701.05517142] |dx|=0.14974298726284502
iter=6 x=[-3889502.39769341  3164362.9459861   3928918.74908853    48701.05517143] |dx|=0.0003646285868578597
position error (ENU) = [ 0.46618346 -0.92739469  6.70778863]
 tcb[estim] = 48701.055 tcb = 48696.649


リスト13（a）

In [ ]:
dops = np.diag(E@Q[0:3,0:3]@E.T)
pdop = np.sqrt(np.sum(np.diag(Q[0:3,0:3])))
tdop = np.sqrt(Q[3,3])
hdop = np.sqrt(dops[0]+dops[1])
vdop = np.sqrt(dops[2])
print(f"PDOP = {pdop:.2f}  HDOP = {hdop:.2f}  VDOP = {vdop:.2f} T DOP = {tdop:.2f}")

PDOP = 4.08  HDOP = 1.42  VDOP = 3.83 T DOP = 2.96
